In [1]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import logging ### to monitor the code
from bs4 import BeautifulSoup
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re

import nltk
from nltk.collocations import *

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import xml.etree.cElementTree as ET
from urllib.request import urlopen

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
from google.oauth2 import service_account # based on google-auth library
import sddk

In [3]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
endpoint variable has been configured to: https://sciencedata.dk/files/


In [4]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get(conf[1] + "ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
PIA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KxOx7Be9fj3lDcEPgQhQ-Iqcn9p367-MMD6RMXe8rks/edit?usp=sharing")
PIA_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1e94wyelg6dftQ4zxbq1xvwxWAI-BhcYXtclDW-YTnrw/edit?usp=sharing")

In [5]:
### our data are located within a publicly available folder on sciencedata.dk
publicfolder = "31b393e2afe1ee96ce81869c7efe18cb"
c_aristotelicum = sddk.read_file("c_aristotelicum.json", "df", publicfolder)
c_hippocraticum = sddk.read_file("c_hippocraticum.json", "df", publicfolder)

reading file located in a public folder
reading file located in a public folder


In [6]:
# look at first 5 rows
c_aristotelicum.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount
685,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora,59614,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Πρῶτον εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις...,3710,"[[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστ...","[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστή...",24680
686,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima,20912,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ὑπολαμβάνοντε...,1317,"[[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτ...","[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτε...",9395
688,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo,30794,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ἡ πλείστη φαίνετ...,2276,"[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε...","[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...",13437
689,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae,10316,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,"ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοινόν, ὁ δὲ κα...",660,"[[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα,...","[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...",4678
690,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum,1194,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Περὶ δὲ τῆς μαντικῆς τῆς ἐν τοῖς ὕπνοις γινομέ...,68,"[[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐ...","[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐν...",549


# Exploring Nicomachean Ethics
feel free to skip this section

In [7]:
EN_df = c_aristotelicum[c_aristotelicum["doc_id"]=="tlg0086.tlg010"]
EN_df

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount
692,tlg0086.tlg010.perseus-grc1.xml,Aristotle,Nicomachean Ethics,56620,tlg0086,tlg0086.tlg010,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,"πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ πρᾶξίς ...",3920,"[[τέχνη, πᾶς, μέθοδος, πρᾶξις, προαίρεσις, ἀγα...","[τέχνη, πᾶς, μέθοδος, πρᾶξις, προαίρεσις, ἀγαθ...",25082


In [8]:
print(EN_df["lemmatized_sentences"].tolist()[0][:10])

[['τέχνη', 'πᾶς', 'μέθοδος', 'πρᾶξις', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι', 'δοκέω'], ['ἀποφαίνω', 'ἀγαθός', 'πᾶς', 'ἐφίημι'], ['διαφορά', 'φαίνω', 'τέλος'], ['εἰμί', 'ἐνέργεια', 'αὐτός', 'ἔργον'], ['τέλος', 'πρᾶξις', 'βελτίων', 'φύω', 'ἐνέργεια', 'ἔργον'], ['πολύς', 'πρᾶξις', 'τέχνη', 'ἐπιστήμη', 'πολύς', 'γίγνομαι', 'τέλος'], ['ἰατρικός', 'ὑγίεια', 'ναυπηγικός', 'πλοῖον', 'στρατηγικός', 'νίκη', 'οἰκονομικός', 'πλοῦτος'], ['τοιοῦτος', 'εἷς', 'δύναμις', 'ἱππικός', 'χαλινοποιική', 'ἱππικός', 'ὄργανον', 'εἰμί', 'πολεμικός', 'πρᾶξις', 'στρατηγικός', 'αὐτός', 'τρόπος', 'ἄλλος', 'ἕτερος'], ['ἅπας', 'ἀρχιτεκτονικῶν', 'τέλος', 'αἱρετός'], ['οὗτος', 'χάρις', 'ἐκεῖνος', 'διώκω']]


In [9]:
EN_df["lemmatized_sentences"].tolist()[0][:5]

[['τέχνη',
  'πᾶς',
  'μέθοδος',
  'πρᾶξις',
  'προαίρεσις',
  'ἀγαθός',
  'ἐφίημι',
  'δοκέω'],
 ['ἀποφαίνω', 'ἀγαθός', 'πᾶς', 'ἐφίημι'],
 ['διαφορά', 'φαίνω', 'τέλος'],
 ['εἰμί', 'ἐνέργεια', 'αὐτός', 'ἔργον'],
 ['τέλος', 'πρᾶξις', 'βελτίων', 'φύω', 'ἐνέργεια', 'ἔργον']]

In [10]:
" ".join(EN_df["lemmatized_sentences"].tolist()[0][0])

'τέχνη πᾶς μέθοδος πρᾶξις προαίρεσις ἀγαθός ἐφίημι δοκέω'

In [11]:
EN_sentences = ""
for sentence in EN_df["lemmatized_sentences"].tolist()[0]:
  EN_sentences = EN_sentences + " ".join(sentence) + ". "

In [12]:
EN_sentences[:1000]

'τέχνη πᾶς μέθοδος πρᾶξις προαίρεσις ἀγαθός ἐφίημι δοκέω. ἀποφαίνω ἀγαθός πᾶς ἐφίημι. διαφορά φαίνω τέλος. εἰμί ἐνέργεια αὐτός ἔργον. τέλος πρᾶξις βελτίων φύω ἐνέργεια ἔργον. πολύς πρᾶξις τέχνη ἐπιστήμη πολύς γίγνομαι τέλος. ἰατρικός ὑγίεια ναυπηγικός πλοῖον στρατηγικός νίκη οἰκονομικός πλοῦτος. τοιοῦτος εἷς δύναμις ἱππικός χαλινοποιική ἱππικός ὄργανον εἰμί πολεμικός πρᾶξις στρατηγικός αὐτός τρόπος ἄλλος ἕτερος. ἅπας ἀρχιτεκτονικῶν τέλος αἱρετός. οὗτος χάρις ἐκεῖνος διώκω. διαφέρω ἐνέργεια τέλος πρᾶξις οὗτος ἄλλος λεχθεισῶν ἐπιστήμη. τέλος πρακτός ἑαυτοῦ βούλομαι ἄλλος πᾶς ἕτερος αἱρέω ʽπρόεισι ἀπείρων εἰμί κενός μάταιος ὄρεξις δῆλος οὗτος ἀγαθός ἀγαθός. βίος γνῶσις μέγας ἔχω ῥοπή τοξότης σκοπός ἔχω τυγχάνω δέον. πειρατέος τύπος περιλαμβάνω ποτός ἐπιστήμη δύναμις. δοκέω κύριος ἀρχιτεκτονικῆς. τοιοῦτος πολιτικός φαίνω. ἐπιστήμη πόλις ποῖος ἕκαστος μανθάνω οὗτος διατάσσω. ὁράω ἔντιμος δύναμις εἰμί οἷος στρατηγικός οἰκονομικός ῥητορικός. χρωμένης οὗτος λοιπός πρακτικός ἐπιστήμη νομοθετούσ

In [13]:
file_object = open("EN_sentences.txt","w") 
file_object.write(EN_sentences)

185257

In [14]:
print(EN_df["lemmata"].tolist()[0][:100])

['τέχνη', 'πᾶς', 'μέθοδος', 'πρᾶξις', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι', 'δοκέω', 'ἀποφαίνω', 'ἀγαθός', 'πᾶς', 'ἐφίημι', 'διαφορά', 'φαίνω', 'τέλος', 'εἰμί', 'ἐνέργεια', 'αὐτός', 'ἔργον', 'τέλος', 'πρᾶξις', 'βελτίων', 'φύω', 'ἐνέργεια', 'ἔργον', 'πολύς', 'πρᾶξις', 'τέχνη', 'ἐπιστήμη', 'πολύς', 'γίγνομαι', 'τέλος', 'ἰατρικός', 'ὑγίεια', 'ναυπηγικός', 'πλοῖον', 'στρατηγικός', 'νίκη', 'οἰκονομικός', 'πλοῦτος', 'τοιοῦτος', 'εἷς', 'δύναμις', 'ἱππικός', 'χαλινοποιική', 'ἱππικός', 'ὄργανον', 'εἰμί', 'πολεμικός', 'πρᾶξις', 'στρατηγικός', 'αὐτός', 'τρόπος', 'ἄλλος', 'ἕτερος', 'ἅπας', 'ἀρχιτεκτονικῶν', 'τέλος', 'αἱρετός', 'οὗτος', 'χάρις', 'ἐκεῖνος', 'διώκω', 'διαφέρω', 'ἐνέργεια', 'τέλος', 'πρᾶξις', 'οὗτος', 'ἄλλος', 'λεχθεισῶν', 'ἐπιστήμη', 'τέλος', 'πρακτός', 'ἑαυτοῦ', 'βούλομαι', 'ἄλλος', 'πᾶς', 'ἕτερος', 'αἱρέω', 'ʽπρόεισι', 'ἀπείρων', 'εἰμί', 'κενός', 'μάταιος', 'ὄρεξις', 'δῆλος', 'οὗτος', 'ἀγαθός', 'ἀγαθός', 'βίος', 'γνῶσις', 'μέγας', 'ἔχω', 'ῥοπή', 'τοξότης', 'σκοπός', 'ἔχω', 'τυγχάνω', '

# Key term replacement

In [38]:
### produce a list of all words from the authors
### (useful for a preliminary exploration)
aristotle_list = []
for list_element in c_aristotelicum["lemmata"].tolist():
  aristotle_list.extend(list_element)
hippocrates_list = []
for list_element in c_hippocraticum["lemmata"].tolist():
  hippocrates_list.extend(list_element)
one_merged_list = aristotle_list + hippocrates_list

In [65]:
# manually define key terms we are interested in:
keyterm_patterns = [("^λ[υ|ύ]π.+", "λύπ*"), ("[α|ά|ἀ|ἄ]λγ.+", "ἄλγ*"), ("^[ὀ|ὠ]δ[ύ|υ]ν.", "ὀδύν*"), ("^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+", "πόνο*")]

In [66]:
# unique word forms
matches = []
for pattern_tuple in keyterm_patterns:
  r = re.compile(pattern_tuple[0])
  matches.extend([(pattern_tuple[0], pattern_tuple[1], match, aristotle_list.count(match), hippocrates_list.count(match)) for match in list(filter(r.search, list(set(aristotle_list + hippocrates_list))))])
matches_df = pd.DataFrame(matches, columns=["pattern", "replacement", "match", "c_aristotelicum", "c_hippocraticum"]) #, "translation"])
matches_df

,pattern,replacement,match,c_aristotelicum,c_hippocraticum
0,^λ[υ|ύ]π.+,λύπ*,λυπέουσιν,0,1
1,^λ[υ|ύ]π.+,λύπ*,λυπέον,0,8
2,^λ[υ|ύ]π.+,λύπ*,λυπούμενον,4,0
3,^λ[υ|ύ]π.+,λύπ*,λυπέῃ,0,5
4,^λ[υ|ύ]π.+,λύπ*,λυπητικός,1,0
...,...,...,...,...,...
157,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονέον,0,8
158,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονεούσης,0,1
159,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονέω,46,105
160,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονήσασιν,0,1


In [67]:
matches_ch = matches_df[matches_df["c_hippocraticum"]>0]
matches_ch

,pattern,replacement,match,c_aristotelicum,c_hippocraticum
0,^λ[υ|ύ]π.+,λύπ*,λυπέουσιν,0,1
1,^λ[υ|ύ]π.+,λύπ*,λυπέον,0,8
3,^λ[υ|ύ]π.+,λύπ*,λυπέῃ,0,5
7,^λ[υ|ύ]π.+,λύπ*,λυπέωσι,0,1
8,^λ[υ|ύ]π.+,λύπ*,λυπέω,113,16
...,...,...,...,...,...
157,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονέον,0,8
158,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονεούσης,0,1
159,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονέω,46,105
160,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονήσασιν,0,1


In [68]:
### look at all unique matched words
print(matches_df["match"].tolist())

['λυπέουσιν', 'λυπέον', 'λυπούμενον', 'λυπέῃ', 'λυπητικός', 'λυπουμένου', 'λυπεῖταἰ', 'λυπέωσι', 'λυπέω', 'λύπημα', 'λυπεόντων', 'λυπηρός', 'λυπέοιτο', 'λυπέοντος', 'λυποδίαν', 'λύπαςμεσότης', 'λυπέοντα', 'λυπρᾷ', 'λύπη', 'λυπέουσαν', 'λυπέεται', 'λυπούμενα', 'λυπέουσα', 'λυπηθῆ', 'λύπαςοὐ', 'λυπέονταί', 'κεφαλαλγίῃσι', 'κεφαλαλγικοὶ', 'κεφαλαλγικοὺς', 'κεφαλαλγία', 'ὑπεραλγέω', 'ὐσφυαλγέσιν', 'ὀσφυαλγέες', 'ἀναλγησία', 'ἀλγέω', 'προαλγήσαντι', 'ἀλγέωσι', 'ὀσφυαλγίην', 'κεφαλαλγικοῖσιν', 'ἀναλγέα', 'καρδιαλγέες', 'καρδιαλγικὰ', 'ἀλγέοντας', 'ὀσφυαλγέσιν', 'ἀλγέοντάς', 'λαλγίην', 'κεφαλαλγίης', 'κεφαλαλγίῃσιν', 'συναλγοῦντας', 'ἄλγλος', 'ʽἀλγεῖ', 'ἀλγηδών', 'ἀλγέοντι', 'προαλγοῦσι', 'κεφαλαλγίην', 'ὀσφυαλγήσασι', 'ἀλγέουσαν', 'ἀλγήσωσιν', 'ἀλγέειν', 'ἀλγέοντος', 'καρδιαλγικός', 'ἄλγος', 'ἀλγοῦντες', 'ἀλγέον', 'ἀλγεῦντα', 'κεφαλαλγήσει', 'καρδιαλγίης', 'ἀνάλγητος', 'ἀλγέῃ', 'ἀλγήσῃ', 'κεφαλαλγίαι', 'ὑπεραλγοῦντας', 'ἀλγήμασι', 'ἀλγεινός', 'ἀλγέουσι', 'προαλγήσαντες', 'κεφαλαλγικῷ', 'ἀλγο

In [64]:
matches_ch[matches_ch["replacement"] ==  "ἄλγ*" ].sort_values("c_hippocraticum", ascending=False).head(20)

,pattern,replacement,match,c_aristotelicum,c_hippocraticum
54,^[ἀ|ἄ]λγ.+,ἄλγ*,ἄλγημα,3,141
26,^[ἀ|ἄ]λγ.+,ἄλγ*,ἀλγέω,14,84
41,^[ἀ|ἄ]λγ.+,ἄλγ*,ἀλγέῃ,0,25
31,^[ἀ|ἄ]λγ.+,ἄλγ*,ἀλγηδών,7,12
45,^[ἀ|ἄ]λγ.+,ἄλγ*,ἀλγέουσι,0,10
37,^[ἀ|ἄ]λγ.+,ἄλγ*,ἄλγος,4,7
35,^[ἀ|ἄ]λγ.+,ἄλγ*,ἀλγέειν,0,5
47,^[ἀ|ἄ]λγ.+,ἄλγ*,ἀλγέουσιν,0,4
48,^[ἀ|ἄ]λγ.+,ἄλγ*,ἀλγέοντα,0,3
50,^[ἀ|ἄ]λγ.+,ἄλγ*,ἀλγεόντων,0,2


In [53]:
for key_word in matches_ch["replacement"].unique():
    print(matches_ch[matches_ch["replacement"] == key_word].sort_values("c_hippocraticum", ascending=False))

       pattern replacement      match  c_aristotelicum  c_hippocraticum
8   ^λ[υ|ύ]π.+        λύπ*      λυπέω              113               16
18  ^λ[υ|ύ]π.+        λύπ*       λύπη              197               11
1   ^λ[υ|ύ]π.+        λύπ*     λυπέον                0                8
3   ^λ[υ|ύ]π.+        λύπ*      λυπέῃ                0                5
11  ^λ[υ|ύ]π.+        λύπ*    λυπηρός               83                3
16  ^λ[υ|ύ]π.+        λύπ*   λυπέοντα                0                3
20  ^λ[υ|ύ]π.+        λύπ*   λυπέεται                0                3
0   ^λ[υ|ύ]π.+        λύπ*  λυπέουσιν                0                1
7   ^λ[υ|ύ]π.+        λύπ*    λυπέωσι                0                1
9   ^λ[υ|ύ]π.+        λύπ*     λύπημα                0                1
10  ^λ[υ|ύ]π.+        λύπ*  λυπεόντων                0                1
12  ^λ[υ|ύ]π.+        λύπ*   λυπέοιτο                0                1
13  ^λ[υ|ύ]π.+        λύπ*  λυπέοντος                0          

In [19]:
# uncomment the following to export to gsheet: 
set_with_dataframe(PIA_overview.add_worksheet("matches_20210106", 1, 1), matches_df)

In [20]:
# make 
def replacer_word_list(pattern, product, word_list):
  return [re.sub(pattern, product, word) for word in word_list]

def replace_keywords(list_of_words, list_of_tuples):
  for pattern in list_of_tuples:
    list_of_words = replacer_word_list(pattern[0], pattern[1], list_of_words)
  return list_of_words

In [21]:
# test 1:
testlist = ["μέλυπρᾷ", "λυπρᾷ"] 
for word in testlist:
  print(re.sub(keyterm_patterns[0][0], keyterm_patterns[0][1], word))

μέλυπρᾷ
λύπ*


In [22]:
### test 2:
word_list_test = ['βοοκ', 'πᾶς', 'μέλυπρᾷ', "ἄλγτέχνη",'τέχνη' ,'πᾶς', 'μέθοδος', 'ὅμοιος', "λύπη",'πρᾶξίς', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι']
replace_keywords(word_list_test, keyterm_patterns)

['βοοκ',
 'πᾶς',
 'μέλυπρᾷ',
 'ἄλγ*',
 'τέχνη',
 'πᾶς',
 'μέθοδος',
 'ὅμοιος',
 'λύπ*',
 'πρᾶξίς',
 'προαίρεσις',
 'ἀγαθός',
 'ἐφίημι']

In [23]:
# apply the replacement on the level of individual words
c_aristotelicum["lemmata_repl"] = c_aristotelicum["lemmata"].apply(lambda x: replace_keywords(x, keyterm_patterns))
c_hippocraticum["lemmata_repl"] = c_hippocraticum["lemmata"].apply(lambda x: replace_keywords(x, keyterm_patterns))

In [24]:
# apply the replacement on the sentences
def replace_in_sentences(list_of_sentences):
    return [replace_keywords(sentence, keyterm_patterns) for sentence in list_of_sentences]

c_aristotelicum["lemmatized_sentences_repl"] = c_aristotelicum["lemmatized_sentences"].apply(replace_in_sentences)
c_hippocraticum["lemmatized_sentences_repl"] = c_hippocraticum["lemmatized_sentences"].apply(replace_in_sentences)

In [25]:
c_aristotelicum.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount,lemmata_repl,lemmatized_sentences_repl
685,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora,59614,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Πρῶτον εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις...,3710,"[[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστ...","[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστή...",24680,"[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστή...","[[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστ..."
686,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima,20912,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ὑπολαμβάνοντε...,1317,"[[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτ...","[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτε...",9395,"[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτε...","[[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτ..."
688,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo,30794,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ἡ πλείστη φαίνετ...,2276,"[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε...","[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...",13437,"[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...","[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε..."
689,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae,10316,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,"ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοινόν, ὁ δὲ κα...",660,"[[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα,...","[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...",4678,"[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...","[[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα,..."
690,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum,1194,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Περὶ δὲ τῆς μαντικῆς τῆς ἐν τοῖς ὕπνοις γινομέ...,68,"[[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐ...","[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐν...",549,"[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐν...","[[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐ..."


In [26]:
sddk.write_file("public_data/PIA/c_aristotelicum_repl.json", c_aristotelicum, conf)
sddk.write_file("public_data/PIA/c_hippocraticum_repl.json", c_hippocraticum, conf)

A file with the same name ("c_aristotelicum_repl.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/public_data/PIA/c_aristotelicum_repl.json"
A file with the same name ("c_hippocraticum_repl.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/public_data/PIA/c_hippocraticum_repl.json"
